In [28]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras import regularizers

In [29]:
df = pd.read_json('/home/leviathan/Downloads/News_Category_Dataset.json',lines=True)

In [30]:
df.drop(['date','link'],axis=1,inplace=True)

In [31]:
df.columns

Index(['authors', 'category', 'headline', 'short_description'], dtype='object')

In [32]:
df_down = df[df.category=='POLITICS']
df_down = df_down[:3500]
df  = df[df.category != 'POLITICS']

In [33]:
df = pd.concat([df,df_down])

In [34]:
df_down = df[df.category=='ENTERTAINMENT']
df_down = df_down[:3500]
df  = df[df.category != 'ENTERTAINMENT']
df = pd.concat([df,df_down])

In [35]:
df = shuffle(df)
df.shape

(84993, 4)

In [36]:
X = df.drop('category',axis=1)
Y = df.category
print(X.shape,Y.shape)

(84993, 3) (84993,)


In [39]:
# Y['output'] = df['category'].factorize()[0]
Y.output

array([ 0,  1,  2, ...,  4, 27,  9])

In [11]:
X['data'] = X.authors+" "+X.headline
X = X['data']
print(X.shape)

(84993,)


In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=0,test_size=0.2)

In [14]:
X_train.shape

(67994,)

In [15]:
words=5000
max_len = 150
tok = Tokenizer(num_words=words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
data = sequence.pad_sequences(sequences,maxlen=max_len)
word_index = tok.word_index

In [16]:
# embeddings_index = {}
# f = open(os.path.join('/home/leviathan/Downloads/glove.6B', 'glove.6B.100d.txt'))
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [17]:
# embedding_matrix = np.zeros((len(word_index) + 1, 100))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [18]:
model = Sequential()
model.add(Embedding(words, 100, input_length=max_len))
# model.add(Embedding(len(word_index) + 1,100,weights=[embedding_matrix],input_length=max_len,trainable=False))
model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))
# model.add(Dense(31,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(31, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          5844800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 31)                3131      
Total params: 5,928,331
Trainable params: 83,531
Non-trainable params: 5,844,800
_________________________________________________________________


In [19]:
model.fit(data, Y_train, validation_split=0.2, epochs=10, batch_size=128)

Train on 54395 samples, validate on 13599 samples
Epoch 1/10
54395/54395 [==============================] - 176s 3ms/step - loss: 3.0617 - acc: 0.1453 - val_loss: 2.5386 - val_acc: 0.2849
Epoch 2/10
54395/54395 [==============================] - 176s 3ms/step - loss: 2.6163 - acc: 0.2629 - val_loss: 2.1489 - val_acc: 0.3947
Epoch 3/10
54395/54395 [==============================] - 180s 3ms/step - loss: 2.3676 - acc: 0.3327 - val_loss: 1.9480 - val_acc: 0.4447
Epoch 4/10
54395/54395 [==============================] - 182s 3ms/step - loss: 2.2225 - acc: 0.3732 - val_loss: 1.8231 - val_acc: 0.4844
Epoch 5/10
54395/54395 [==============================] - 184s 3ms/step - loss: 2.1278 - acc: 0.3999 - val_loss: 1.7291 - val_acc: 0.5092
Epoch 6/10
54395/54395 [==============================] - 186s 3ms/step - loss: 2.0489 - acc: 0.4240 - val_loss: 1.6737 - val_acc: 0.5253
Epoch 7/10
54395/54395 [==============================] - 186s 3ms/step - loss: 1.9897 - acc: 0.4414 - val_loss: 1.6206 - 

In [20]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [21]:
accr = model.evaluate(test_sequences_matrix,Y_test)

16999/16999 [==============================] - 24s 1ms/step


In [22]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 1.546
  Accuracy: 0.556
